In [0]:

SELECT 
    'STG_SMA_CASE_PHASE_PROPERTIES' as table_name,
    :p_process_key as process_key, 
    'MAIN' as source_sys_origin, 
    count(1) as INSERT_CNT,
    0 as UPDATE_CNT
FROM gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES
;

SELECT 
    'STG_SMA_CASE_PHASE_PROPERTIES_THECASE' as table_name,
    :p_process_key as process_key, 
    'MAIN' as source_sys_origin, 
    count(1) as INSERT_CNT,
    0 as UPDATE_CNT
FROM gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_THECASE
;

select 
    'SMA_CASE_PHASE_PROPERTIES' as table_name,
    :p_process_key as process_key, 
    'MAIN' as source_sys_origin, 
    count(1) as INSERT_CNT,
    0 as UPDATE_CNT
FROM gap_catalog.ads_owner.SMA_CASE_PHASE_PROPERTIES
;

SELECT
        'PROCESS_EVENTS' as table_name,
        :p_process_key as process_key, 
        'MAIN' as source_sys_origin, 
        SUM(CASE WHEN PROCE_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN PROCE_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
FROM gap_catalog.ads_owner.PROCESS_EVENTS
WHERE (PROCE_UPDATE_PROCESS_KEY = :p_process_key OR PROCE_INSERT_PROCESS_KEY = :p_process_key)
;
--11m19sec


select 
        'CASE_PHASE_PROPERTIES' as table_name,
        :p_process_key as process_key, 
        'MAIN' as source_sys_origin, 
        SUM(CASE WHEN CPPROP_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN CPPROP_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
WHERE (CPPROP_UPDATE_PROCESS_KEY = :p_process_key OR CPPROP_INSERT_PROCESS_KEY = :p_process_key)
AND CPPTP_KEY <> 225
AND CTP_KEY <> 1429
AND CPPTP_KEY NOT IN (
    select CPPTP_KEY FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES CPPTP
    WHERE CPPTP.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
                        AND CPPTP.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
                        AND CPPTP.CPPTP_VALID_TO= DATE'3000-01-01'); 

select 
        'CASE_PHASE_PROPERTIES' as table_name,
        :p_process_key as process_key, 
        'HUMANTASK' as source_sys_origin, 
        SUM(CASE WHEN CPPROP_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN CPPROP_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
WHERE (CPPROP_UPDATE_PROCESS_KEY = :p_process_key OR CPPROP_INSERT_PROCESS_KEY = :p_process_key)
AND CPPTP_KEY IN (
    select CPPTP_KEY FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES CPPTP
    WHERE CPPTP.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
                        AND CPPTP.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
                        AND CPPTP.CPPTP_VALID_TO= DATE'3000-01-01'); 

select 
        'CASE_PHASE_PROPERTIES' as table_name,
        :p_process_key as process_key, 
        'POT_OWN' as source_sys_origin, 
        SUM(CASE WHEN CPPROP_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN CPPROP_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
WHERE (CPPROP_UPDATE_PROCESS_KEY = :p_process_key OR CPPROP_INSERT_PROCESS_KEY = :p_process_key)
AND CPPTP_KEY = 225;

select 
        'CASE_PHASE_PROPERTIES' as table_name,
        :p_process_key as process_key, 
        'BUSY_BANKING' as source_sys_origin, 
        SUM(CASE WHEN CPPROP_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN CPPROP_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
FROM gap_catalog.ads_owner.CASE_PHASE_PROPERTIES
WHERE (CPPROP_UPDATE_PROCESS_KEY = :p_process_key OR CPPROP_INSERT_PROCESS_KEY = :p_process_key)
AND CTP_KEY = 1429;

describe gap_catalog.ads_owner.CASE_PHASE_PROPERTIES;
------------------------------------------------------------
MERGE INTO gap_catalog.log.performance_test_log AS target
USING (
    -- EVENT_TYPES table
    SELECT 
        'EVENT_TYPES' as table_name,
        :p_process_key as process_key, 
        evetp_source_sys_origin as source_sys_origin, 
        SUM(CASE WHEN EVETP_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN EVETP_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
    FROM gap_catalog.ads_owner.event_types
    WHERE (EVETP_UPDATE_PROCESS_KEY = :p_process_key OR EVETP_INSERT_PROCESS_KEY = :p_process_key)
    GROUP BY evetp_source_sys_origin
    /*UNION ALL
    -- EVENT_STATUS table
    SELECT 
        'EVENT_STATUS' as table_name,
        :p_process_key as process_key, 
        est_source_sys_origin as source_sys_origin, 
        SUM(CASE WHEN EST_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN EST_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
    FROM gap_catalog.ads_owner.event_status
    WHERE (EST_UPDATE_PROCESS_KEY = :p_process_key OR EST_INSERT_PROCESS_KEY = :p_process_key)
    GROUP BY est_source_sys_origin
    
    UNION ALL
    
    -- CASE_OBJECT_STATUS table
    SELECT 
        'CASE_OBJECT_STATUS' as table_name,
        :p_process_key as process_key, 
        cos_source_sys_origin as source_sys_origin, 
        SUM(CASE WHEN COS_INSERT_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as INSERT_CNT,
        SUM(CASE WHEN COS_UPDATE_PROCESS_KEY = :p_process_key THEN 1 ELSE 0 END) as UPDATE_CNT
    FROM gap_catalog.ads_owner.case_object_status
    WHERE (COS_UPDATE_PROCESS_KEY = :p_process_key OR COS_INSERT_PROCESS_KEY = :p_process_key)
    GROUP BY cos_source_sys_origin
*/    
) AS src
ON target.process_key = src.process_key 
   AND target.target_table = src.table_name 
   AND target.source_sys_origin = src.source_sys_origin
WHEN MATCHED AND (target.target_insert_count IS NULL AND target.target_update_count IS NULL) THEN 
    UPDATE SET 
        target_insert_count = src.INSERT_CNT,
        target_update_count = src.UPDATE_CNT,
        updated_at = CURRENT_TIMESTAMP();